In [1]:
%matplotlib inline
import glob

from pandas import DataFrame, read_csv

import pandas as pd

#Versioning
import sys

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 28 days
/home/alexdiep/anaconda3/lib/python3.5/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)

Python version 3.5.1 |Anaconda 2.5.0 (64-bit)| (default, Dec  7 2015, 11:16:01) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Pandas version 0.18.0


In [3]:
search_words = ['non-coding',
                'intergenic',
                'intron',
                'exon',
                'promoter-TSS',
                'TTS',
                '5\' UTR',
                '3\' UTR',
                'p53',
                'rpr',
                'corp']
search_words = [word.lower() for word in search_words]

In [4]:
files = glob.glob('data/*.anno')
genes = [read_csv(f, sep='\t') for f in files]

for gene in genes:
    # Fix the PeakID label b/c it had 'junk' in it
    gene.columns.values[0] = 'PeakID'

In [5]:
# Drop 'data/' for only the filenames
files = [file.split('/')[1] for file in files]

In [6]:
# Drop all data beyond 'Gene Type'
genes = [gene.ix[:,:'Gene Type'] for gene in genes]

In [7]:
# Gene data indexed by filenames and number of row in file
genes_data = pd.concat(genes, keys=files)

In [8]:
gene_names = genes_data.groupby(level=0)['Gene Name'].value_counts()
gene_names.groupby(level=0).head()

                       Gene Name         
SP53NT60_B.anno        CG40228               26
                       CG17684               21
                       CR43242               15
                       CG42598               14
                       CR44043               14
R_G085_P53_NT60.anno   Cadps                  4
                       CG14621                2
                       CR43941                2
                       CR45139                2
                       uex                    2
SP53XR30.anno          CR43242               14
                       scro                  12
                       CG14621               11
                       CG40228               11
                       RYa                   11
Botcheve_IMR90.anno    CDKN1A                 3
                       GDF15                  3
                       IRF2BPL                3
                       KLLN                   3
                       SUGCT                  

In [9]:
# Table contains all of the rows with the gene 'CDKN1A'
genes_data[genes_data['Gene Name'].str.contains('CDKN1A', na=False)]

PeakID   Chr     Start       End Strand  \
Botcheve_IMR90.anno     11          36752168  chr6  36751791  36752590      +   
                        97          36753075  chr6  36752686  36753373      +   
                        231         36758405  chr6  36758122  36758684      +   
Akdemir_p53_DOX.anno    37    MACS_peak_3944  chr6  36751522  36756745      +   
                        2586  MACS_peak_3945  chr6  36758106  36759518      +   
Akdemir_p53_Untr.anno   38    MACS_peak_3558  chr6  36751654  36752444      +   
Younger_Human_ChIP.anno 977             1964  chr6  36634693  36635391      +   
                        2301            1965  chr6  36643697  36646241      +   
                        2511            1966  chr6  36649603  36651324      +   

                             Peak Score Focus Ratio/Region Size  \
Botcheve_IMR90.anno     11          114                     NaN   
                        97           30                     NaN   
                        231          18                     NaN   
Akdemir_p53_DOX.anno    37         3100                     NaN   
                        2586     145.35                     NaN   
Akdemir_p53_Untr.anno   38       935.46                     NaN   
Younger_Human_ChIP.anno 977           0                     NaN   
                        2301          0                     NaN   
                        2511          0                     NaN   

                                                        Annotation  \
Botcheve_IMR90.anno     11                promoter-TSS (NM_078467)   
                        97    intron (NM_001220777, intron 1 of 2)   
                        231   intron (NM_001220778, intron 1 of 2)   
Akdemir_p53_DOX.anno    37                promoter-TSS (NM_000389)   
                        2586  intron (NM_001220778, intron 1 of 2)   
Akdemir_p53_Untr.anno   38                promoter-TSS (NM_078467)   
Younger_Human_ChIP.anno 977                             Intergenic   
                        2301  intron (NM_001220777, intron 1 of 2)   
                        2511  intron (NM_001220778, intron 1 of 2)   

                                               Detailed Annotation  \
Botcheve_IMR90.anno     11                                     NaN   
                        97                                     NaN   
                        231                                    NaN   
Akdemir_p53_DOX.anno    37                promoter-TSS (NM_000389)   
                        2586  intron (NM_001220778, intron 1 of 2)   
Akdemir_p53_Untr.anno   38                promoter-TSS (NM_078467)   
Younger_Human_ChIP.anno 977                             Intergenic   
                        2301  intron (NM_001220777, intron 1 of 2)   
                        2511  intron (NM_001220778, intron 1 of 2)   

                             Distance to TSS Nearest PromoterID Entrez ID  \
Botcheve_IMR90.anno     11               -25       NM_001220777      1026   
                        97               784          NM_078467      1026   
                        231             3938       NM_001220778      1026   
Akdemir_p53_DOX.anno    37              -301          NM_000389      1026   
                        2586            4347       NM_001220778      1026   
Akdemir_p53_Untr.anno   38              -166       NM_001220777      1026   
Younger_Human_ChIP.anno 977            -9195       NM_001220777      1026   
                        2301             702          NM_078467      1026   
                        2511            3976       NM_001220778      1026   

                             Nearest Unigene Nearest Refseq  Nearest Ensembl  \
Botcheve_IMR90.anno     11         Hs.732576   XM_005248787  ENSG00000124762   
                        97         Hs.732576   XM_005248787  ENSG00000124762   
                        231        Hs.732576   XM_005248787  ENSG00000124762   
Akdemir_p53_DOX.anno    37         Hs.732576   XM_005248787  EN

In [10]:
# Takes the list of search_words and gives a table of the hits and files

# Create a boolean mask based on whether the cell contains the keyword then 'collapse' them to a single column
hits_mask = [genes_data.applymap(lambda x, word=word: word in str(x).lower()).any(axis='columns') for word in search_words]

# Apply the mask on the data table and get the size of the results as the count
# .count().iloc[:, 0] is a hack for .size()
# .size() causes werid issues
hits_results = [genes_data[mask].groupby(level=0).count().iloc[:, 0] for mask in hits_mask]

# Group the data by search_words and fill in NaN as 0
search_data = pd.DataFrame(hits_results, index=search_words).fillna(0)

In [13]:
search_data.index.name = 'Search Word'; search_data

,Akdemir_p53_DOX.anno,Akdemir_p53_Untr.anno,Botcheve_IMR90.anno,Kc167_P53_NT60_A.anno,Kc167_P53_NT60_B.anno,Kc167_P53_XR60_A.anno,Kc167_P53_XR60_B.anno,R_G085_P53_NT60.anno,R_G085_P53_XR60.anno,R_P53NT60.anno,...,SP53XR30.anno,SP53XR30Ctr.anno,SP53XR60Ctr.anno,SP53XR60_A.anno,SP53XR60_B.anno,S_P53NT120.anno,S_P53NT60AB.anno,S_P53XR120.anno,S_P53XR60AB.anno,Younger_Human_ChIP.anno
Search Word,,,,,,,,,,,,,,,,,,,,,
non-coding,38.0,34.0,9.0,2.0,4.0,2.0,2.0,0.0,0.0,3.0,...,1.0,0.0,1.0,1.0,0.0,3.0,1.0,1.0,0.0,10.0
intergenic,2379.0,1940.0,246.0,58.0,140.0,44.0,104.0,10.0,8.0,482.0,...,269.0,28.0,75.0,1053.0,221.0,375.0,713.0,325.0,221.0,1337.0
intron,1898.0,1453.0,233.0,78.0,160.0,68.0,108.0,2.0,2.0,289.0,...,152.0,30.0,71.0,612.0,167.0,206.0,815.0,219.0,167.0,1145.0
exon,262.0,308.0,85.0,20.0,66.0,16.0,60.0,0.0,0.0,34.0,...,9.0,1.0,7.0,33.0,22.0,18.0,40.0,14.0,22.0,75.0
promoter-tss,423.0,802.0,176.0,102.0,172.0,80.0,144.0,2.0,0.0,56.0,...,39.0,8.0,24.0,76.0,48.0,33.0,120.0,60.0,48.0,72.0
tts,66.0,66.0,8.0,22.0,90.0,14.0,86.0,2.0,2.0,52.0,...,26.0,8.0,13.0,50.0,53.0,27.0,99.0,40.0,53.0,31.0
5' utr,66.0,105.0,30.0,14.0,4.0,10.0,4.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,10.0
3' utr,44.0,43.0,6.0,2.0,12.0,0.0,8.0,0.0,0.0,8.0,...,1.0,0.0,1.0,6.0,4.0,3.0,12.0,2.0,4.0,22.0
p53,32.0,18.0,10.0,3.0,3.0,3.0,3.0,1.0,1.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,19.0


In [12]:
search_data.to_csv('search_results.tsv', sep='\t')